In [1]:
from sudoku_stuff import *

# Resolviendo Sudokus con Algoritmos Genéticos
Inteligencia Artificial - Facundo A. Lucianna - CEIA - FIUBA

En las notebooks anteriores, intentamos resolver Sudokus utilizando los algoritmos de gradiente descendente, simulated annealing y búsqueda local beam. Vimos que los mejores resultados los obtuvimos con **búsqueda local beam**. Ahora veamos cómo nos va con los **algoritmos genéticos**.

Como vimos en los videos, el algoritmo genético ya no se considera un enfoque puramente de búsqueda local, sino que introduce los conceptos de reproducción y mutación. Los nuevos estados que se generen serán el resultado de la combinación entre sus "padres".

Resolvamos este problema:

<div>
<img src="./sudoku_7.png" width="300"/>
</div>

El diccionario de **celdas fijas** queda de la siguiente forma:

In [2]:
fixed_squares = {
    'A1': 3, 'A3': 4, 'A4': 5, 'A5': 6, 'A7': 9,
    'B1': 1, 'B2': 8, 'B3': 5, 'B6': 9, 'B7': 7,
    'C5': 7, 'C6': 8, 'C7': 4, 'C8': 1, 'C9': 5,
    'D2': 2, 'D5': 1, 'D8': 4, 'D9': 9,
    'E2': 4, 'E3': 9, 'E5': 5, 
    'F3': 1, "F4": 9, "F5": 8, "F7": 6, "F8": 7,
    'G1': 4, 'G2': 9, 'G5': 3, 'G9': 7, 
    'H2': 1, 'H3': 8, 'H4': 7, 'H5': 4, 'H6': 5, 'H9': 6,
    'I8': 8,
}

Y la solución que tenemos es:

In [3]:
solution = {
    'A1': 3, 'A2': 7, 'A3': 4, 'A4': 5, 'A5': 6, 'A6': 1, 'A7': 9, 'A8': 2, 'A9': 8,
    'B1': 1, 'B2': 8, 'B3': 5, 'B4': 4, 'B5': 2, 'B6': 9, 'B7': 7, 'B8': 6, 'B9': 3,
    'C1': 9, 'C2': 6, 'C3': 2, 'C4': 3, 'C5': 7, 'C6': 8, 'C7': 4, 'C8': 1, 'C9': 5,
    'D1': 8, 'D2': 2, 'D3': 7, 'D4': 6, 'D5': 1, 'D6': 3, 'D7': 5, 'D8': 4, 'D9': 9,
    'E1': 6, 'E2': 4, 'E3': 9, 'E4': 2, 'E5': 5, 'E6': 7, 'E7': 8, 'E8': 3, 'E9': 1,
    'F1': 5, 'F2': 3, 'F3': 1, 'F4': 9, 'F5': 8, 'F6': 4, 'F7': 6, 'F8': 7, 'F9': 2,
    'G1': 4, 'G2': 9, 'G3': 6, 'G4': 8, 'G5': 3, 'G6': 2, 'G7': 1, 'G8': 5, 'G9': 7,
    'H1': 2, 'H2': 1, 'H3': 8, 'H4': 7, 'H5': 4, 'H6': 5, 'H7': 3, 'H8': 9, 'H9': 6,
    'I1': 7, 'I2': 5, 'I3': 3, 'I4': 1, 'I5': 9, 'I6': 6, 'I7': 2, 'I8': 8, 'I9': 4,
}

In [4]:
print_state(solution)

*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*


## Implementando herramientas para aplicar Algoritmos Genéticos en la resolución de Sudokus

Un algoritmo genético es una variante estocástica de la búsqueda local beam, en la cual los estados sucesores se generan combinando dos estados padres (reproducción). Para implementar este algoritmo, debemos definir varios elementos:

- La forma de codificar un estado como una cadena (cromosoma).
- La función de reproducción.
- La función de mutación.
- La función de idoneidad (fitness), que en este caso será la función de costo que venimos utilizando.

### Cromosoma

Primero, veamos cómo podemos codificar un estado particular del Sudoku como un cromosoma, que será una cadena de caracteres que luego podremos manipular para realizar reproducción y mutación.

En este caso, dado que un Sudoku está formado por números, representaremos todos los valores que completan el Sudoku como una única cadena, aplanando las filas en una secuencia lineal.

Veamos un ejemplo de implementación utilizando la solución definida previamente:

In [5]:
# Las funciones que implementan los distintos elementos del algoritmo genético están definidas en genetic.py
from genetic import *

In [6]:
print_state(solution)

*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*


In [7]:
# obtain_chromosome_sudoku devuelve el cromosoma del Sudoku dado un estado (formado por un diccionario con las celdas)
chromosome_solution = obtain_chromosome_sudoku(solution, obtain_all_cells())

In [8]:
print(chromosome_solution)

374561928185429763962378415827613549649257831531984672496832157218745396753196284


A continuación, necesitamos una función que, al recibir un cromosoma, nos devuelva el estado correspondiente. Es decir, el proceso de "nacimiento" de nuevos estados. Para esto, utilizaremos la función `obtain_sibling_from_chromosome_sudoku()`.

Antes de eso, veamos cómo identificar las posiciones fijas dentro del cromosoma:

In [9]:
pos_fixed = obtain_fixed_pos_in_chromosome_sudoku(fixed_squares, obtain_all_cells())

print(pos_fixed)

[0, 2, 3, 4, 6, 9, 10, 11, 14, 15, 22, 23, 24, 25, 26, 28, 31, 34, 35, 37, 38, 40, 47, 48, 49, 51, 52, 54, 55, 58, 62, 64, 65, 66, 67, 68, 71, 79]


Con esta función podemos asegurarnos de que, al aplicar mutaciones, dichas posiciones fijas no sean modificadas.

Finalmente, tomemos el cromosoma que calculamos anteriormente y modifiquemos el número en el índice `1:`

In [10]:
new_chromosome = chromosome_solution[:1] + '1' + chromosome_solution[2:]
print(new_chromosome)

314561928185429763962378415827613549649257831531984672496832157218745396753196284


Ahora obtengamos el estado correspondiente a ese cromosoma modificado:

In [11]:
new_state = obtain_sibling_from_chromosome_sudoku(new_chromosome, obtain_all_cells())
print_state(new_state)

*---------+---------+---------*
| 3  1  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*


### Reproducción

La reproducción la realizamos mediante la siguiente operación: dados dos cromosomas, seleccionamos un punto de corte aleatorio en la cadena. A partir de ese punto, concatenamos una parte del cromosoma de un padre con la del otro. Esta reproducción generará dos hijos, siendo cada uno el complemento del otro.

Esta operación está implementada en la función `reproduction_sudoku()`. Veamos un ejemplo de uso:

In [12]:
# Generamos dos estados al azar 
state_1 = init_state(fixed_squares)
state_2 = init_state(fixed_squares)

print("Estado 1")
print_state(state_1)
print("Estado 2")
print_state(state_2)

Estado 1
*---------+---------+---------*
| 3  6  4 | 5  6  8 | 9  8  9 |
| 1  8  5 | 7  9  9 | 7  3  3 |
| 9  4  4 | 4  7  8 | 4  1  5 |
*---------+---------+---------*
| 3  2  3 | 9  1  1 | 1  4  9 |
| 9  4  9 | 7  5  8 | 2  8  9 |
| 8  5  1 | 9  8  9 | 6  7  7 |
*---------+---------+---------*
| 4  9  1 | 5  3  8 | 6  7  7 |
| 6  1  8 | 7  4  5 | 3  1  6 |
| 4  9  6 | 3  1  6 | 5  8  3 |
*---------+---------+---------*
Estado 2
*---------+---------+---------*
| 3  4  4 | 5  6  4 | 9  1  8 |
| 1  8  5 | 3  7  9 | 7  1  9 |
| 5  7  8 | 7  7  8 | 4  1  5 |
*---------+---------+---------*
| 6  2  1 | 7  1  8 | 5  4  9 |
| 5  4  9 | 6  5  8 | 1  5  4 |
| 2  4  1 | 9  8  3 | 6  7  5 |
*---------+---------+---------*
| 4  9  3 | 4  3  3 | 6  9  7 |
| 8  1  8 | 7  4  5 | 6  1  6 |
| 1  4  7 | 3  1  2 | 6  8  9 |
*---------+---------+---------*


In [13]:
# Obtenemos sus cromosomas
chromosome_state_1 = obtain_chromosome_sudoku(state_1, obtain_all_cells())
chromosome_state_2 = obtain_chromosome_sudoku(state_2, obtain_all_cells())

print("Estado 1")
print(chromosome_state_1)
print("Estado 2")
print(chromosome_state_2)

Estado 1
364568989185799733944478415323911149949758289851989677491538677618745316496316583
Estado 2
344564918185379719578778415621718549549658154241983675493433697818745616147312689


In [14]:
# Realizamos la reproducción
chromosome_sibling_1, chromosome_sibling_2 = reproduction_sudoku(chromosome_state_1, chromosome_state_2)

print("Estado hijo 1")
print(chromosome_sibling_1)
print("Estado hijo 2")
print(chromosome_sibling_2)

Estado hijo 1
364568988185379719578778415621718549549658154241983675493433697818745616147312689
Estado hijo 2
344564919185799733944478415323911149949758289851989677491538677618745316496316583


In [15]:
# Obtenemos los hijos
sibling_1 = obtain_sibling_from_chromosome_sudoku(chromosome_sibling_1, obtain_all_cells())
sibling_2 = obtain_sibling_from_chromosome_sudoku(chromosome_sibling_2, obtain_all_cells())

print("Estado hijo 1")
print_state(sibling_1)
print("Estado hijo 2")
print_state(sibling_2)

Estado hijo 1
*---------+---------+---------*
| 3  6  4 | 5  6  8 | 9  8  8 |
| 1  8  5 | 3  7  9 | 7  1  9 |
| 5  7  8 | 7  7  8 | 4  1  5 |
*---------+---------+---------*
| 6  2  1 | 7  1  8 | 5  4  9 |
| 5  4  9 | 6  5  8 | 1  5  4 |
| 2  4  1 | 9  8  3 | 6  7  5 |
*---------+---------+---------*
| 4  9  3 | 4  3  3 | 6  9  7 |
| 8  1  8 | 7  4  5 | 6  1  6 |
| 1  4  7 | 3  1  2 | 6  8  9 |
*---------+---------+---------*
Estado hijo 2
*---------+---------+---------*
| 3  4  4 | 5  6  4 | 9  1  9 |
| 1  8  5 | 7  9  9 | 7  3  3 |
| 9  4  4 | 4  7  8 | 4  1  5 |
*---------+---------+---------*
| 3  2  3 | 9  1  1 | 1  4  9 |
| 9  4  9 | 7  5  8 | 2  8  9 |
| 8  5  1 | 9  8  9 | 6  7  7 |
*---------+---------+---------*
| 4  9  1 | 5  3  8 | 6  7  7 |
| 6  1  8 | 7  4  5 | 3  1  6 |
| 4  9  6 | 3  1  6 | 5  8  3 |
*---------+---------+---------*


### Mutación

Un aspecto importante en los algoritmos genéticos es la aleatoriedad introducida por la mutación. La idea es que, al azar, se modifique una parte del cromosoma, lo que podría llevar a encontrar un estado que se adapte mejor a la solución. En este caso, al mutar, se cambia una posición del cromosoma —que no sea fija— por un valor aleatorio.

Para decidir si se realiza la mutación, usamos una **“temperatura”**, que determina la probabilidad de mutar: cuanto más alta sea, mayor será esa probabilidad; cuanto más baja, menor será. Implementamos un mecanismo similar al de simulated annealing, donde se acepta la mutación si un valor aleatorio entre 0 y 1 es menor que `exp(-1 * (1 / temperatura))`.

Esto está implementado en la función `mutate_chromosome_sudoku_with_temperature()`. Veamos un ejemplo utilizando uno de los hijos generados anteriormente, con una temperatura muy alta para forzar la mutación:

In [16]:
print("Estado hijo 1")
print_state(sibling_1)

Estado hijo 1
*---------+---------+---------*
| 3  6  4 | 5  6  8 | 9  8  8 |
| 1  8  5 | 3  7  9 | 7  1  9 |
| 5  7  8 | 7  7  8 | 4  1  5 |
*---------+---------+---------*
| 6  2  1 | 7  1  8 | 5  4  9 |
| 5  4  9 | 6  5  8 | 1  5  4 |
| 2  4  1 | 9  8  3 | 6  7  5 |
*---------+---------+---------*
| 4  9  3 | 4  3  3 | 6  9  7 |
| 8  1  8 | 7  4  5 | 6  1  6 |
| 1  4  7 | 3  1  2 | 6  8  9 |
*---------+---------+---------*


In [17]:
# Realizamos la mutación
chromosome_mutated = mutate_chromosome_sudoku_with_temperature(chromosome_sibling_1, pos_fixed, temperature=100000)

In [18]:
# Obtenemos al hijo mutado
sibling_mutated = obtain_sibling_from_chromosome_sudoku(chromosome_mutated, obtain_all_cells())

print("Estado mutado")
print_state(sibling_mutated)

Estado mutado
*---------+---------+---------*
| 3  6  4 | 5  6  8 | 9  8  8 |
| 1  8  5 | 3  7  9 | 7  1  9 |
| 5  7  8 | 7  7  8 | 4  1  5 |
*---------+---------+---------*
| 6  2  1 | 7  1  5 | 5  4  9 |
| 5  4  9 | 6  5  8 | 1  5  4 |
| 2  4  1 | 9  8  3 | 6  7  5 |
*---------+---------+---------*
| 4  9  3 | 4  3  3 | 6  9  7 |
| 8  1  8 | 7  4  5 | 6  1  6 |
| 1  4  7 | 3  1  2 | 6  8  9 |
*---------+---------+---------*


## Implementación de Búsqueda con Algoritmos Genéticos

Con todos los elementos necesarios definidos e implementados para el caso particular de Sudokus, ahora podemos armar nuestra implementación de búsqueda utilizando algoritmos genéticos. La última pieza que nos faltaba era la función de idoneidad para seleccionar los mejores individuos, pero para eso ya contamos con nuestra función de costo.

> 🧠 **Nota**: Todas las decisiones tomadas para el diseño —como la forma de reproducción, la codificación del cromosoma, etc.— son arbitrarias y dependen de la creatividad del diseñador. Otras decisiones pueden dar resultados muy diferentes.

Veamos la implementación, leyendo cada comentario con atención:

In [19]:
import itertools

In [20]:
def genetic_algorithm_sudoku(initial_generation: list, fixed_squares: dict, max_iterations: int = 50, initial_temperature: float = 100) -> tuple:
    """
    Realiza la optimización del Sudoku utilizando un algoritmo genético.

    Args:
        initial_generation (list): Lista con estados iniciales del Sudoku.
        fixed_squares (dict): Diccionario que contiene las casillas fijas del Sudoku.
        max_iterations (int, optional): Número máximo de iteraciones permitidas. Por defecto es 50.
        initial_temperature (float, optional): Temperatura inicial para controlar la probabilidad de mutación. Por defecto es 100.

    Returns:
        dict: El mejor estado encontrado después de la optimización.
        float: El costo del mejor estado encontrado.
        int: Número que indica en qué generación se encontró el mejor resultado.
    """
    temperature = initial_temperature
    best_state = initial_generation[0]
    best_cost = cost_function(best_state)
    best_iteration = 0
    no_changes = 0

    # Determinamos el tamaño de la población
    number_population = len(initial_generation)
    current_population = initial_generation.copy()

    # Obtenemos las posiciones del Sudoku que no pueden mutar
    squares = obtain_all_cells()
    not_valid_positions = obtain_fixed_pos_in_chromosome_sudoku(fixed_squares, squares)

    # Iteramos hasta alcanzar el número máximo de generaciones
    for iteration in range(max_iterations):

        no_changes += 1

        # Calculamos el costo de cada estado en la generación actual
        actual_cost_list = [cost_function(state) for state in current_population]

        generate_print = False
        for index, cost in enumerate(actual_cost_list):
            # Actualizamos el mejor estado si encontramos uno con menor costo
            if cost < best_cost:
                no_changes = 0
                generate_print = True
                best_state = current_population[index]
                best_cost = cost
                best_iteration = iteration

        # Si encontramos un estado con costo 0, hemos encontrado la solución.
        # Si pasan muchas generaciones sin mejoras, asumimos estancamiento y detenemos la búsqueda.
        if best_cost == 0 or no_changes > 9:
            return best_state, best_cost, best_iteration

        if generate_print:
            print(f"El mejor costo es: {best_cost} en la iteración {iteration}")

        # Ordenamos los estados por su función de costo (idoneidad)
        index_list = sorted(range(len(actual_cost_list)), key=lambda x: actual_cost_list[x])
        current_population = [current_population[k] for k in index_list]

        # Conservamos solo la mejor parte de la generación para reproducirse
        current_population = current_population[:number_population]

        # Obtenemos los cromosomas de los estados actuales
        all_chromosome = [obtain_chromosome_sudoku(state, squares) for state in current_population]

        # Generamos los hijos: todos se reproducen con todos
        sibling_chromosomes_list = []
        for chromosome_1, chromosome_2 in itertools.combinations(all_chromosome, 2):
            offspring_1, offspring_2 = reproduction_sudoku(chromosome_1, chromosome_2)
            sibling_chromosomes_list.extend([offspring_1, offspring_2])

        # Aplicamos mutaciones a los hijos
        sibling_chromosomes_list = [mutate_chromosome_sudoku_with_temperature(chromosome, not_valid_positions, temperature)
                                    for chromosome in sibling_chromosomes_list]

        # Diezmamos la población actual: solo el 10% sobrevive
        current_population = current_population[:number_population // 10]

        # Creamos la nueva generación a partir de los cromosomas hijos y los agregamos a la población diezmada
        current_population += [obtain_sibling_from_chromosome_sudoku(chromosome, squares) for chromosome in
                               sibling_chromosomes_list]

    # Si se alcanzó el máximo de iteraciones, devolvemos el mejor resultado encontrado
    return best_state, best_cost, best_iteration

Ahora, veamos si podemos encontrar la solución en una única ejecución. Para ello, llamaremos a una función llamada `execute_search_evolution()`, a la cual le pasamos la implementación del algoritmo. Esta función inicializa varios estados de Sudoku al azar, aplica la búsqueda y verifica si se ha alcanzado una solución o no.

In [21]:
from processing import execute_search_evolution

In [22]:
solution_bool, best_state, best_generation, _ = execute_search_evolution(0, genetic_algorithm_sudoku, fixed_squares, number_generation_initial=200)

El mejor costo es: 5.7 en la iteración 0
El mejor costo es: 5.000000000000002 en la iteración 1
El mejor costo es: 4.55 en la iteración 2
El mejor costo es: 3.800000000000001 en la iteración 3
El mejor costo es: 3.300000000000001 en la iteración 4
El mejor costo es: 2.8000000000000007 en la iteración 5
El mejor costo es: 2.0500000000000003 en la iteración 6
El mejor costo es: 1.8500000000000003 en la iteración 7
El mejor costo es: 1.4000000000000004 en la iteración 8
El mejor costo es: 1.1 en la iteración 9
El mejor costo es: 0.7999999999999999 en la iteración 10


In [23]:
print(f"Valor del mejor costo encontrado: {cost_function(best_state)}. El mejor estado se encontró en la generación {best_generation}.")
print("Mejor estado encontrado:")
print_state(best_state)

Valor del mejor costo encontrado: 0.7999999999999999. El mejor estado se encontró en la generación 10.
Mejor estado encontrado:
*---------+---------+---------*
| 3  7  4 | 5  6  3 | 9  8  2 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 1  7  8 | 4  1  5 |
*---------+---------+---------*
| 6  2  3 | 8  1  7 | 5  4  9 |
| 7  4  9 | 3  5  6 | 1  2  8 |
| 8  5  1 | 9  8  2 | 6  7  3 |
*---------+---------+---------*
| 4  9  6 | 2  3  1 | 8  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 5  3  7 | 6  9  4 | 2  8  1 |
*---------+---------+---------*


In [24]:
print("¿El estado encontrado es una solución?")
if solution_bool:
    print("El estado que encontramos verifica correctamente que es la solución.")
else:
    print("El estado que encontramos no es una solución.")

¿El estado encontrado es una solución?
El estado que encontramos no es una solución.


Vemos que no se encontró la solución. El algoritmo estuvo muy cerca, alcanzando un costo bajo y utilizando pocas generaciones, pero llegó a un punto en el que solo un par de números estaban mal ubicados (con solo 4 penalizaciones en las filas 5 y 6). El problema en esta implementación es que, rápidamente, los estados de bajo costo dominan las generaciones, y para salir de ese estado solo sería posible mediante una mutación que modifique justo algunos de esos valores. Sin embargo, esto es poco probable, ya que solo se modifica un valor y, aunque este se ubique correctamente, podría seguir generando la misma penalización.

Se podrían implementar dos mejoras, que se dejan como ejercicio:

- Implementar un segundo tipo de mutación en el que, en lugar de cambiar un valor al azar, se intercambien dos valores de posición.
- Cada cierta cantidad de generaciones, introducir nuevos estados completamente aleatorios o permitir que sobrevivan estados con alto costo, con el fin de introducir más variedad en la población.

Como prueba final, vamos a ejecutar la búsqueda 100 veces y verificar cuántas veces se llega a una solución. Para acelerar el proceso, aprovecharemos que contamos con **CPUs multinúcleo**.

Para ello, llamaremos a la función `parallel_sudoku_search()`, a la cual le pasaremos la función de búsqueda y el número de iteraciones que queremos realizar, indicando que estamos usando un algoritmo que opera con múltiples estados.

In [25]:
from processing import parallel_sudoku_search

# Debemos llamar a la función de búsqueda desde un archivo .py, ya que los threads no pueden recibir la función directamente desde la notebook.
from search_methods import genetic_algorithm_sudoku

In [26]:
results = parallel_sudoku_search(genetic_algorithm_sudoku, fixed_squares, max_iterations=100, generation_method=True, number_generation_initial=200)

  0%|          | 0/100 [00:00<?, ?it/s]

Veamos si algún proceso encontró la solución:

In [27]:
show_solution = True
for res in results:
    # Acá se devuelve un booleano que indica si se encontró la solución
    is_solution = res[0]
    # Este es el último estado encontrado en esta iteración
    last_state = res[1]
    # Este es el estado desde donde partió la búsqueda
    initial_state = res[2]
    # Este es el identificador de la iteración en la que se obtuvo la solución
    process_id = res[-1]

    if is_solution:
        if show_solution:
            print_state(last_state)
            show_solution = False
        print(f"En la iteración {process_id} se encontró la solución.")

*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*
En la iteración 28 se encontró la solución.
En la iteración 88 se encontró la solución.


Vemos que, de las 100 ejecuciones, solo dos vez se encontró la solución, mientras que en el caso de **Local Beam Search**, los resultados fueron mucho más exitosos. La forma en que se generan los vecinos resulta más efectiva que los métodos de reproducción y mutación implementados aquí.

Viendo el lado positivo, este comportamiento revela en qué aspectos es fuerte este algoritmo. Rápidamente encuentra mínimos, incluso más profundos que otros algoritmos cuando estos fallan. Sin embargo, no es tan eficiente para encontrar el mejor caso, como en el Sudoku, donde solo existe una única solución. Si el problema fuera más general —por ejemplo, cuando buscamos un estado que cumpla con ciertos criterios de optimización, como minimizar un parámetro, error, energía o costo por debajo de un umbral— este algoritmo sería más útil.

Eso sí, siempre con la desventaja del gran trabajo de diseño previo que requiere.